In [ ]:
from datasets import SOCOFingGender
from FPMLmodule.backbones import ResNet50, MobileNetV2
from FPMLmodule.classifiers import ResNetClassifier, DefaultClassifier, AlexNetClassifier, MobileNetClassifier
import FPMLmodule.utils as utils
from FPMLmodule.fpml import FPML 
from tensorflow.keras.optimizers import Adam, Nadam, RMSprop
from tensorflow.keras.models import Sequential
import matplotlib.pylab as plt
import numpy as np

import tensorflow as tf
print("Tensorflow version " + tf.__version__)

In [ ]:
# Global Config
seed=9
img_dim = (120, 120, 3)
img_height, img_width, img_channels = img_dim
batch_size = 32

# Dataset configuration

AUTOTUNE = tf.data.AUTOTUNE
split_ratio = [0.7, 0.15, 0.15]
shuffle=True
dsConfig = {
    'batchSize': batch_size, 
    'parallelTune': AUTOTUNE, 
    'split': split_ratio, 
    'inputDim': img_dim, 
    'seed': seed, 
    'shuffle': shuffle
    }
weightsRN50 = "./weights/resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5"
weightsMNV2 = "./weights/mobilenet_v2_weights_tf_dim_ordering_tf_kernels_1.0_224_no_top.h5"

# Model Config
learning_rate = 0.001
epochs_find_best = 10
epochs_best = 100

In [ ]:
SOCOGender = SOCOFingGender(**dict(dsConfig, sampling=SOCOFingGender.UNDER_SAMPLING))
genderDs = SOCOGender.create()
utils.displayDsSplit(genderDs)
train_ds, test_ds, val_ds = genderDs
# Show split by gender


In [ ]:
nbClasses = len(SOCOGender.classNames)
activation = "softmax"

classifiers = {
    "ResNet50" : ResNetClassifier(nbClasses, activation),
    "AlexNet" : AlexNetClassifier(nbClasses, activation),
    "Default" : DefaultClassifier(nbClasses, activation), 
    "MobileNetV2" : MobileNetClassifier(nbClasses, activation)
}

backbones = {
    "ResNet50" : ResNet50(img_dim, weights=weightsRN50, trainable=False),
    "MobileNetV2-pretrained" : MobileNetV2(img_dim, weights=weightsMNV2, trainable=False),
    "MobileNetV2-trainable": MobileNetV2(img_dim, weights=None)
}

optimizers = {
    "Adam": Adam,
    "Nadam": Nadam,
    "RMSProp": RMSprop
}


In [ ]:
hypers = {
    "optimizer": Adam,
    "learningRate": learning_rate,
    "loss": 'binary_crossentropy',
    "metrics": 'accuracy'
}
pretrainedVsFulltrainig = {
    "mobilenetV2-pretrained" : {
        "architecture": {
            "backbone":backbones["MobileNetV2-pretrained"],
            "classfier":classifiers["MobileNetV2"],
            "inputLayer":"", 
            "inputDim": img_dim
        },
        "hyperparameters":hypers
    },
    "MobileNetV2-trainable" : {
        "architecture": {
            "backbone":backbones["MobileNetV2-trainable"],
            "classfier":classifiers["MobileNetV2"],
            "inputLayer":"", 
            "inputDim": img_dim
        },
        "hyperparameters":hypers
    }
}

In [ ]:
studies = {
    "resnet50" : {
        "architecture": {
            "backbone":backbones["ResNet50"],
            "classfier":classifiers["ResNet50"],
            "inputLayer":"", 
            "inputDim": img_dim
        },
        "hyperparameters":{
            "optimizer": Adam,
            "learningRate": learning_rate,
            "loss": 'binary_crossentropy',
            "metrics": 'accuracy'
        }
    },
}

histories = utils.researchStudies(train_ds, val_ds, pretrainedVsFulltrainig, 2)



In [ ]:
print(utils.getBestStudyFromHistories(histories))